# HANDWRITTEN DIGIT CLASSIFICATION USING MNIST DATASET 

In [1]:

from numpy.random import seed
seed(888)
import tensorflow as tf
tf.random.set_seed(404)

In [2]:
import os
import numpy as np
import tensorflow.compat.v1 as tf
from time import strftime
from PIL import Image

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


# Constants

In [3]:
X_TRAIN_PATH='MNIST/digit_xtrain.csv'
X_TEST_PATH='MNIST/digit_xtest.csv'
Y_TRAIN_PATH='MNIST/digit_ytrain.csv'
Y_TEST_PATH='MNIST/digit_ytest.csv'

NR_CLASSES=10
VALIDATION_SIZE=10000

IMAGE_HEIGHT=28
IMAGE_WIDHT=28
CHANNELS=1

LOGGING_DIR='tensorboard_mnist_logs/'

TOTAL_INPUTS=IMAGE_HEIGHT * IMAGE_WIDHT * CHANNELS

# Load dataset

In [4]:
%%time
X_train_all=np.loadtxt(X_TRAIN_PATH,delimiter=',',dtype=int)


Wall time: 58 s


In [5]:
%%time
y_train_all=np.loadtxt(Y_TRAIN_PATH,delimiter=',',dtype=int)

Wall time: 411 ms


In [6]:
X_train_all.shape  # 784 = 28 * 28 * 1  image is greay scale so channel=1

(60000, 784)

In [7]:
y_train_all.shape

(60000,)

In [8]:
%%time
X_test=np.loadtxt(X_TEST_PATH,delimiter=',',dtype=int)

Wall time: 9.71 s


In [9]:
%%time
y_test=np.loadtxt(Y_TEST_PATH,delimiter=',',dtype=int)

Wall time: 74 ms


In [10]:
X_test.shape

(10000, 784)

In [11]:
X_test[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [12]:
y_train_all[0] #labels

5

In [13]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Data pre-processing

In [14]:
values=y_train_all[:5]
values

array([5, 0, 4, 1, 9])

In [15]:
np.eye(10)[values]   #np.eye() makes 10/10 matrix with diagonal elements=1 

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [16]:
np.eye(12)[8]   #creates 12/12 matrix with diagonal elements=1  and prints 8th row

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [17]:
# RE_SCALING image pixels between 0-1
X_train_all, X_test = X_train_all/255.0 , X_test /255.0

In [18]:
X_train_all.shape
X_test.shape


(10000, 784)

In [19]:
X_train_all[1] #values in between 0-1

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [20]:
#one hot encoding ie 3 = [0,0,0,1,0,0,0,0,...]
y_train_all=np.eye(NR_CLASSES)[y_train_all]
y_test=np.eye(NR_CLASSES)[y_test]

In [21]:
y_test.shape

(10000, 10)

In [22]:
y_train_all.shape

(60000, 10)

# creating validation set

In [23]:
X_val=X_train_all[:VALIDATION_SIZE]
y_val=y_train_all[:VALIDATION_SIZE]

X_train=X_train_all[VALIDATION_SIZE:]
y_train=y_train_all[VALIDATION_SIZE:]

In [24]:
X_val.shape

(10000, 784)

In [25]:
X_train.shape

(50000, 784)

In [26]:
X_test.shape

(10000, 784)

## Setup tensorflow graph   (Run all cells below for each batch accuracy and loss)

In [27]:
# defining hyperparameters

n_epochs=50
learning_rate=0.001

n_hidden1=512
n_hidden2=64

In [28]:
X = tf.placeholder(tf.float32, shape=[None,TOTAL_INPUTS],name='X') #NONE because size in batches decided later
Y = tf.placeholder(tf.float32, shape=[None,NR_CLASSES],name='labels') #providing name to visualize result in tensorboard

## setting initial weights and biases for hidden layers

In [29]:
# # FOR FIRST HIDDEN LAYER

# initial_w1=tf.truncated_normal(shape=[TOTAL_INPUTS,512],stddev=0.1,seed=42)
# w1=tf.Variable(initial_value=initial_w1,name='w1')  #storing weights in tensorflow variable

# initial_b1=tf.constant(value=0.0,shape=[n_hidden1])

# b1=tf.Variable(initial_value=initial_b1,name='b1') #storing biases in tensorflow variable

# layer1_in = tf.matmul(X , w1) + b1

# layer1_out = tf.nn.relu(layer1_in)

In [30]:
# # FOR SECOND HIDDEN LAYER

# initial_w2=tf.truncated_normal(shape=[n_hidden1,n_hidden2],stddev=0.1,seed=42)
# w2=tf.Variable(initial_value=initial_w2,name='w2')

# initial_b2=tf.constant(value=0.0,shape=[n_hidden2])
# b2=tf.Variable(initial_value=initial_b2,name='b2')

# layer2_in = tf.matmul(layer1_out, w2) + b2
# layer2_out = tf.nn.relu(layer2_in)

In [31]:
# # FOR OUTPUT LAYER

# initial_w3=tf.truncated_normal(shape=[n_hidden2,NR_CLASSES],stddev=0.1,seed=42)
# w3=tf.Variable(initial_value=initial_w3,name='w3')

# initial_b3=tf.constant(value=0.0,shape=[NR_CLASSES])
# b3=tf.Variable(initial_value=initial_b3,name='b3')

# layer3_in = tf.matmul(layer2_out, w3) + b3
# output = tf.nn.softmax(layer3_in)

In [32]:
#setting up function for layers (all three layers)
def setup_layer(input,weight_dims,bias_dims,name):
    with tf.name_scope(name):
        initial_w=tf.truncated_normal(shape=weight_dims,stddev=0.1,seed=42)
        w=tf.Variable(initial_value=initial_w,name='W')  #storing weights in tensorflow variable
        initial_b=tf.constant(value=0.0,shape=bias_dims)
        b=tf.Variable(initial_value=initial_b,name='B') #storing biases in tensorflow variable
        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            output = tf.nn.softmax(layer_in)
        else:
            output = tf.nn.relu(layer_in)
            
        tf.summary.histogram('weights',w)
        tf.summary.histogram('biases',b)
            
        return output
            
            
            
       


In [33]:
 #MODEL WITHOUT DROPPOUT (99.4 ACCURACY)
    
# layer_1=setup_layer(input=X,weight_dims=[TOTAL_INPUTS,512],bias_dims=[n_hidden1],name='layer_1')
# layer_2=setup_layer(input=layer_1,weight_dims=[n_hidden1,n_hidden2],bias_dims=[n_hidden2],name='layer_2')
# output=setup_layer(input=layer_2,weight_dims=[n_hidden2,NR_CLASSES],bias_dims=[NR_CLASSES],name='out')

# model_name=f'{n_hidden1}-{n_hidden2} LR{learning_rate} E{n_epochs}'

In [34]:
layer_1=setup_layer(input=X,weight_dims=[TOTAL_INPUTS,512],bias_dims=[n_hidden1],name='layer_1')
layer_drop=tf.nn.dropout(layer_1,keep_prob=0.8,name='dropout_layer')
layer_2=setup_layer(input=layer_drop,weight_dims=[n_hidden1,n_hidden2],bias_dims=[n_hidden2],name='layer_2')
output=setup_layer(input=layer_2,weight_dims=[n_hidden2,NR_CLASSES],bias_dims=[NR_CLASSES],name='out')

model_name=f'{n_hidden1}-DROPOUT-{n_hidden2} LR{learning_rate} E{n_epochs}'

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Setting up tensorboard

In [35]:
folder_name=f'{model_name} at {strftime("%d_%m_%Y %H_%M_%S")}'
directory=os.path.join(LOGGING_DIR,folder_name)
try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Succesfully created directory')
    

Succesfully created directory


# Loss , optimization,accuracy

#### Defining loss function

In [36]:
with tf.name_scope('loss_calc'):
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

#### Defining optimizer

In [37]:
with tf.name_scope('optimizer'):
    optimizer=tf.train.AdamOptimizer(learning_rate) 
    train_step=optimizer.minimize(loss) #to update weights after each batch training

#### Defining Accuracy metrics

In [38]:
with tf.name_scope('accuracy_calc'):
    model_prediction=tf.argmax(output,axis=1,name='prediction')
    correct_pred=tf.equal(model_prediction,tf.argmax(Y,axis=1))  #axis=1 row (if predicted = actual)
    accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))   #mean of accuracy of all batches

In [39]:
tf.summary.scalar('accuracy',accuracy)   #accuracy summary
tf.summary.scalar('loss',loss)  #loss summary

<tf.Tensor 'loss:0' shape=() dtype=string>

## Show input images in tensorboard

In [40]:
with tf.name_scope('show_image'):
    X_image=tf.reshape(X,[-1,28,28,1])  # input tensor(X) is flattened we need to reshape to 28*28*1
    tf.summary.image('image_input',X_image,max_outputs=4)

## Run session

In [41]:
sess= tf.Session()

## Setup filewriter and merge summaries

In [42]:
merged_summary=tf.summary.merge_all()
train_writer=tf.summary.FileWriter(directory+'/train')
train_writer.add_graph(sess.graph)

validation_writer=tf.summary.FileWriter(directory+'/validation')

In [43]:
init=tf.global_variables_initializer()
sess.run(init)

In [44]:
# b1.eval(sess)   #bias values after running session (initial)

In [45]:
# b3.eval(sess)

### Batching data

In [46]:
size_of_batch=1000


In [47]:
num_examples = y_train.shape[0]   #50000
nr_iterations = int(num_examples/size_of_batch) #50000/1000=50

index_in_epoch=0   #0-50000

In [48]:
def next_batch(batch_size,data,labels):
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += size_of_batch
    
    if index_in_epoch > num_examples:  #end of dataset(50000) ,begin next epoch
        start=0
        index_in_epoch=batch_size
    
    end=index_in_epoch
    
    
    return data[start:index_in_epoch],labels[start:index_in_epoch]

### Training loop

In [49]:
for epoch in range(n_epochs):
    
    # ========================== Training set =========================================
    
    for i in range(nr_iterations):
        batch_x,batch_y=next_batch(batch_size=size_of_batch,data=X_train,labels=y_train)
        
        feed_dict={X:batch_x,Y:batch_y}  #feeding dict to session
        
        sess.run(train_step,feed_dict=feed_dict)  #minimise training loss of batch
        
    s,batch_accuracy=sess.run(fetches=[merged_summary,accuracy],feed_dict=feed_dict)
        
    print(f'Epoch : {epoch} \t | Training accuracy : {batch_accuracy}')
    
    train_writer.add_summary(s,epoch)
    
    # ========================Validation set ============================================
    
    summary=sess.run(fetches=merged_summary,feed_dict={X:X_val,Y:y_val})
    validation_writer.add_summary(summary,epoch)
    
print('Done Training')
        
        

Epoch : 0 	 | Training accuracy : 0.8519999980926514
Epoch : 1 	 | Training accuracy : 0.859000027179718
Epoch : 2 	 | Training accuracy : 0.8659999966621399
Epoch : 3 	 | Training accuracy : 0.8730000257492065
Epoch : 4 	 | Training accuracy : 0.8709999918937683
Epoch : 5 	 | Training accuracy : 0.8790000081062317
Epoch : 6 	 | Training accuracy : 0.9779999852180481
Epoch : 7 	 | Training accuracy : 0.9789999723434448
Epoch : 8 	 | Training accuracy : 0.9800000190734863
Epoch : 9 	 | Training accuracy : 0.9850000143051147
Epoch : 10 	 | Training accuracy : 0.9850000143051147
Epoch : 11 	 | Training accuracy : 0.9890000224113464
Epoch : 12 	 | Training accuracy : 0.9860000014305115
Epoch : 13 	 | Training accuracy : 0.9850000143051147
Epoch : 14 	 | Training accuracy : 0.9869999885559082
Epoch : 15 	 | Training accuracy : 0.9890000224113464
Epoch : 16 	 | Training accuracy : 0.9890000224113464
Epoch : 17 	 | Training accuracy : 0.9879999756813049
Epoch : 18 	 | Training accuracy : 0.98

## Saving Model

In [50]:
inputs={'X':X}
outputs={'accuracy_calc/prediction':model_prediction}
tf.compat.v1.saved_model.simple_save(sess,'SavedModel',inputs=inputs,outputs=outputs)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: SavedModel\saved_model.pb


## Make a prediction

In [51]:
img=Image.open('MNIST/test_img.png')
img

In [52]:
black_white=img.convert('L') #convertin to black_white coz we trained model in white in black background

In [53]:
image_array=np.invert(black_white)

In [54]:
image_array.shape

(28, 28)

In [55]:
test_img=image_array.ravel()
test_img.shape

(784,)

## Testing and Evaluation

In [56]:
prediction=sess.run(fetches=tf.argmax(output,axis=1),feed_dict={X:[test_img]})


In [57]:
print(f'prediction for test image is {prediction}')

prediction for test image is [2]


### predicting test set accuracy

In [58]:
test_accuracy=sess.run(fetches=accuracy,feed_dict={X:X_test,Y:y_test})

In [59]:
print(f'test set accuracy is : {test_accuracy:0.2%}')

test set accuracy is : 97.64%


## RESET for next run

In [60]:
sess.close()
train_writer.close()
validation_writer.close()
tf.reset_default_graph()
